In [7]:
import asyncio
import json


async def tcp_echo_client(message):
    reader, writer = await asyncio.open_connection("localhost", 8888)

    print(f"Send: {message!r}")
    writer.write(message.encode())
    await writer.drain()

    data = await reader.read(1000)
    
    # print(f"Received: {data.decode()!r}")
    print("Received:")

    print("Close the connection")
    writer.close()
    await writer.wait_closed()
    print(json.loads(data.decode()))
    return json.loads(data.decode())

In [10]:
import threading


for i in range(10):
    th = threading.Thread(
        target=asyncio.run,
        args=(tcp_echo_client("https://ru.wikipedia.org/wiki/Python"),),
    )
    th.start()

Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Received:
Close the connection
{'python': 344, 'в': 313, 'и': 286, 'с': 147, 'на': 144, 'архивировано': 139, 'дата': 129, 'для': 116, 'править': 114}
Received:
Close the connection
{'python': 344, 'в': 313, 'и': 286, 'с': 147, 'на': 144, 'архивировано': 139, 'дата': 129, 'для': 116, 'править': 114}
Received:
Close the connection
{'python': 344, 'в': 313, 'и': 286, 'с': 147, 'на': 144, 'архивировано': 139, 'дата': 129, 'для': 116, 'править': 114}
Received:
Close the connection
{'python': 344, 'в': 313, 'и': 286, 'с': 147, 'на': 144, 'архивирован

In [5]:
await tcp_echo_client("https://ru.wikipedia.org/wiki/Python")

Send: 'https://ru.wikipedia.org/wiki/Python'
Received: '{"python": 344, "\\u0432": 313, "\\u0438": 286, "\\u0441": 147, "\\u043d\\u0430": 144, "\\u0430\\u0440\\u0445\\u0438\\u0432\\u0438\\u0440\\u043e\\u0432\\u0430\\u043d\\u043e": 139, "\\u0434\\u0430\\u0442\\u0430": 129, "\\u0434\\u043b\\u044f": 116, "\\u043f\\u0440\\u0430\\u0432\\u0438\\u0442\\u044c": 114}'
Close the connection


{'python': 344,
 'в': 313,
 'и': 286,
 'с': 147,
 'на': 144,
 'архивировано': 139,
 'дата': 129,
 'для': 116,
 'править': 114}